## 1. Requirements

In [ ]:

import os
import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim

import time

pd.set_option('future.no_silent_downcasting', True)

In [ ]:
start_time = time.time()

## 2. Set Args

In [ ]:
theta = 7
num_epochs = 50
dropout_ratio = 0.5

data_path = 'datas/dd2/FoodConsumption.csv'
mechanism = 'mnar'
method = 'random'

test_size = 0.3
use_cuda = True
batch_size  = 1 # not in the paper

## 3. Prepare Data

In [ ]:
datas = pd.read_csv(data_path, on_bad_lines='skip')
print(datas[:5])

In [ ]:
# Test 
def per_MissVal(df):
    # Calculer le nombre total de valeurs manquantes
    total_mv = df.isnull().sum().sum()
    # Calculer le nombre total de valeurs dans le DataFrame
    total_val = df.size
    # Calculer le pourcentage de valeurs manquantes
    perc = (total_mv / total_val) * 100
    return perc

In [ ]:
print("data missing values", per_MissVal(datas), "%")

In [ ]:
# ---------------- Detect column types ----------------
numeric_features = datas.select_dtypes(include=["int64", "float64"]).columns.to_list()
categorical_features = datas.select_dtypes(include=["object", "category"]).columns.to_list()

# ---------------- Define numeric imputer using HistGradientBoostingRegressor ----------------
# Custom imputer using HistGradientBoostingRegressor for numeric data
class HGBRImputer:
    def __init__(self):
        self.models = {}
        self.features = None

    # Add y=None to match sklearn's fit signature
    def fit(self, X, y=None):
        self.features = X.columns
        for col in self.features:
            is_missing = X[col].isnull()
            if is_missing.sum() == 0:
                continue
            train_idx = ~is_missing
            # Fill missing values in predictors with median for training
            X_train = X.loc[train_idx].drop(columns=col).fillna(X.loc[train_idx].median())
            y_train = X.loc[train_idx, col]
            model = HistGradientBoostingRegressor(
                learning_rate = 0.1, 
                max_depth = 15, 
                min_samples_leaf = 30, 
                l2_regularization = 0.1, 
                max_bins = 255, 
                scoring = 'neg_mean_squared_error',
                max_iter=100,
                max_leaf_nodes=31,
                random_state=42,
                early_stopping=True,
                validation_fraction=0.25,
                n_iter_no_change=10
            )
            model.fit(X_train, y_train)
            self.models[col] = model
        return self

    def transform(self, X):
        X_filled = X.copy()
        for col, model in self.models.items():
            missing_idx = X_filled[col].isnull()
            if missing_idx.sum() == 0:
                continue
            # Fill missing predictor features with median before prediction
            X_pred = X_filled.loc[missing_idx].drop(columns=col).fillna(X_filled.median())
            preds = model.predict(X_pred)
            X_filled.loc[missing_idx, col] = preds
        return X_filled


# ---------------- Define transformers ----------------

numeric_transformer = Pipeline(steps=[
    ("imputer", HGBRImputer()),
    ("scaler", MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),  # handle missing categorical
    ("ordinal", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

# ---------------- Column Transformer ----------------
transformers = []
if len(numeric_features) > 0:
    transformers.append(("num", numeric_transformer, numeric_features))
if len(categorical_features) > 0:
    transformers.append(("cat", categorical_transformer, categorical_features))

preprocessor = ColumnTransformer(
    transformers=transformers,
    sparse_threshold=0
)

# ---------------- Full Pipeline ----------------
pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

# ---------------- Apply on dataset ----------------
# pipeline.fit(datas)
processed = pipeline.fit_transform(datas)

# Convert back to DataFrame with column names
all_features = []
if len(numeric_features) > 0:
    all_features.extend(numeric_features)
if len(categorical_features) > 0:
    all_features.extend(categorical_features)

df_processed = pd.DataFrame(processed, columns=all_features)

print(df_processed.head())


In [ ]:
data = df_processed.values #datas.values

if use_cuda and not torch.cuda.is_available():
    print("---"* 20)
    print("CUDA is not available. Switching to CPU.")
    use_cuda = False


rows, cols = data.shape
shuffled_index = np.random.permutation(rows)
train_index = shuffled_index[:int(rows*(1-test_size))]
test_index = shuffled_index[int(rows*(1-test_size)):]


train_data = data[train_index, :]
test_data = data[test_index, :]

print("")
print(train_data[:5])
print("")
print(test_data[:5])

In [ ]:
# Exemple d'utilisation
#df = pd.read_csv('data/dd3/titanic.csv')
df0 = pd.DataFrame(train_data)  
df1 = pd.DataFrame(test_data)
#df = df.drop(columns=['Cabin'])
print("train_data missing values", per_MissVal(df0), "%", "--", "test_data missing values", per_MissVal(df1), "%")

In [ ]:
print("number of rows and columns", data.shape)
print()
print("size of the dataset", data.size)

In [ ]:
def missing_method(raw_data, mechanism='mcar', method='uniform') :
    
    data = raw_data.copy()
    rows, cols = data.shape
    
    # missingness threshold
    t = 0.1
    
    if mechanism == 'mcar' :
    
        if method == 'uniform' :
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where v<=t
            mask = (v<=t)
            data[mask] = 0

        elif method == 'random' :
            # only half of the attributes to have missing value
            missing_cols = np.random.choice(cols, cols//2)
            c = np.zeros(cols, dtype=bool)
            c[missing_cols] = True

            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where v<=t
            mask = (v<=t)*c
            data[mask] = 0

        else :
            print("Error : There are no such method")
            raise
    
    elif mechanism == 'mnar' :
        
        if method == 'uniform' :
            # randomly sample two attributes
            sample_cols = np.random.choice(cols, 2)

            # calculate then median m1, m2
            m1, m2 = np.median(data[:,sample_cols], axis=0)
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = data[:,sample_cols[0]] <= m1
            m2 = data[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]

            mask = m*(v<=t)
            data[mask] = 0


        elif method == 'random' :
            # only half of the attributes to have missing value
            missing_cols = np.random.choice(cols, cols//2)
            c = np.zeros(cols, dtype=bool)
            c[missing_cols] = True

            # randomly sample two attributes
            sample_cols = np.random.choice(cols, 2)

            # calculate ther median m1, m2
            m1, m2 = np.median(data[:,sample_cols], axis=0)
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))

            # missing values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = data[:,sample_cols[0]] <= m1
            m2 = data[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]

            mask = m*(v<=t)*c
            data[mask] = 0

        else :
            print("Error : There is no such method")
            raise
    
    else :
        print("Error : There is no such mechanism")
        raise
        
    return data, mask

In [ ]:
missed_data, mask = missing_method(test_data, mechanism=mechanism, method=method)

missed_data = torch.from_numpy(missed_data).float()
train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True)

## 4. Define Model

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
print(f"Using device: {device}")

# Vérifier si le device est bien CUDA ou CPU
if device.type == "cuda":
    print(f"CUDA Device Count: {torch.cuda.device_count()}")
    print(f"Current CUDA Device: {torch.cuda.current_device()}")
    print(f"CUDA Device Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA not available, using CPU.")

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, dim):
        super(Autoencoder, self).__init__()
        self.dim = dim
        
        self.drop_out = nn.Dropout(p=0.5)
        
        self.encoder = nn.Sequential(
            nn.Linear(dim+theta*0, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*3)
        )
            
        self.decoder = nn.Sequential(
            nn.Linear(dim+theta*3, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*0)
        )
        
    def forward(self, x):
        x = x.view(-1, self.dim)
        x_missed = self.drop_out(x)
        
        z = self.encoder(x_missed)
        out = self.decoder(z)
        
        out = out.view(-1, self.dim)
        
        return out

In [ ]:
model = Autoencoder(dim=cols).to(device)

## 5. Define Loss and Optimizer

In [ ]:
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), momentum=0.99, lr=0.01, nesterov=True)

## 6. Train Model

In [ ]:

cost_list = []
early_stop = False

for epoch in range(num_epochs):
    
    total_batch = len(train_data) // batch_size
    
    for i, batch_data in enumerate(train_loader):
        
        batch_data = batch_data.to(device)
        
        reconst_data = model(batch_data)
        cost = loss(reconst_data, batch_data)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
                
        if (i+1) % (total_batch//2) == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.6f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
            
        # early stopping rule 1 : MSE < 1e-06
        if cost.item() < 1e-06 :
            early_stop = True
            break
            
#         early stopping rule 2 : simple moving average of length 5
#         sometimes it doesn't work well.
#         if len(cost_list) > 5 :
#            if cost.item() > np.mean(cost_list[-5:]):
#                early_stop = True
#                break
                
        cost_list.append(cost.item())

    if early_stop :
        break
        
print("Learning Finished!")

## 7. Test Model

In [ ]:
model.eval()
#filled_data = model(missed_data.to(device))
#filled_data = filled_data.cpu().detach().numpy()

with torch.no_grad():   # safer: avoid gradients
    filled_data = model(missed_data.to(device))
    filled_data = filled_data.cpu().detach().numpy()

rmse_sum = 0
num_valid_columns = 0

for i in range(cols) :
    if mask[:,i].sum() > 0 :
        y_actual = test_data[:,i][mask[:,i]]
        y_predicted = filled_data[:,i][mask[:,i]]

        rmse = sqrt(mean_squared_error(y_actual, y_predicted))
        rmse_sum += rmse
        num_valid_columns += 1
    
#print("RMSE_SUM :", rmse_sum)
print(f'RMSE_SUM: {rmse_sum:.3f}')
average_rmse = rmse_sum / num_valid_columns
print("-----------")
print(f'Average RMSE: {average_rmse:.3f}')

In [ ]:
from sklearn.metrics import mean_squared_error
#from numpy import sqrt

mse_sum = 0
num_valid_columns = 0  # To count columns with valid data

for i in range(cols):
    if mask[:, i].sum() > 0:  # Check if there are any missing values
        y_actual = test_data[:, i][mask[:, i]]
        y_predicted = filled_data[:, i][mask[:, i]]

        # Calculate RMSE for the current column
        mse = mean_squared_error(y_actual, y_predicted)
        mse_sum += mse
        num_valid_columns += 1  # Increment valid column count

# Calculate average RMSE if there are valid columns
if num_valid_columns > 0:
    #print("MSE sum:", mse_sum)
    print(f'MSE (sum) Score: {mse_sum:.3f}')
    print("-------")
    average_mse = mse_sum / num_valid_columns
    #print("Average MSE:", average_mse)
    print(f'Average MSE: {average_mse:.3f}')
else:
    print("No valid columns to calculate MSE.")

In [ ]:
from sklearn.metrics import mean_absolute_error

mae_sum = 0
num_valid_columns = 0

for i in range(cols):
    if mask[:, i].sum() > 0:
        y_actual = test_data[:, i][mask[:, i]]
        y_predicted = filled_data[:, i][mask[:, i]]

        mae = mean_absolute_error(y_actual, y_predicted)
        mae_sum += mae
        num_valid_columns += 1

print(f'MAE (sum) Score: {mae_sum:.3f}')
print("------------")
average_mae = mae_sum / num_valid_columns 
print(f'Average MAE Score: {average_mae:.3f}')

In [ ]:
from sklearn.metrics import accuracy_score

acc_sum = 0
count = 0

for i in range(cols):
    if mask[:, i].sum() > 0:  # only evaluate where ground truth exists
        y_actual = test_data[:, i][mask[:, i]]
        y_predicted = filled_data[:, i][mask[:, i]]
        
        # Convert continuous values to discrete labels by rounding or thresholding
        # Here we round values to nearest integer as an example
        y_actual_discrete = np.rint(y_actual).astype(int)
        y_predicted_discrete = np.rint(y_predicted).astype(int)

        # Now compute accuracy_score on discrete labels
        acc = accuracy_score(y_actual_discrete, y_predicted_discrete)
        acc_sum += acc
        count += 1

if count > 0:
    avg_acc = acc_sum / count
    print(f'Accuracy Score (sum): {acc_sum:.3f}')
    print(f'Average Accuracy Score: {avg_acc:.3f}')
else:
    print("No valid columns for Accuracy calculation.")


In [ ]:
print("The time used to execute this is given below")

end_time = time.time()

print(end_time - start_time)